In [82]:
import json
import requests
import time
import csv

API_URL = 'http://api.llama.fi/'

headers = {
    'Accept': '*/*',
    'User-Agent': """Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"""
}

def defillama(endpoint, params={}, json={}):
    resp = requests.get(
        API_URL + endpoint,
        headers=headers
    )
    return resp.json()


data_protocols = json.loads(defillama(f'protocol))

data_api = {}
problematic_slugs = []
for item in data_protocols:
    slug = item['slug']
    
    try:
        protocol_data = defillama(f'protocol/{slug}')
        time.sleep(2) 
        data_api[slug] = protocol_data
    
    except Exception as e:
        print(f"Error scrapping data for slug: {slug}. Error: {e}")
        problematic_slugs.append(slug)
    
print(f"Total problematic slugs: {len(problematic_slugs)}")
                                      
with open('protocol-slug_data.json', 'w') as f:
    json.dump(data_api, f, indent=4)
                                      
with open ('defillama_protocols.json', 'w') as g:
    json.dump(data_protocols, g, indent=4)



Total problematic slugs: 0


In [3]:
import json
import csv
from datetime import datetime

def timestamp_to_date(timestamp):
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d')

with open('protocol-slug_data.json', 'r') as g:
    data_date = json.load(g)
with open('defillama_protocols.json', 'r') as f:
    data_protocols = json.load(f)
    
results = []
problematic_slugs = []
for item in data_protocols:
    protocol_tvl = item['tvl']
    
    if protocol_tvl >= 1000000:
        protocol_name = item['name']
        protocol_address = item['address']
        protocol_slug = item['slug']
        protocol_chains = '|'.join(item['chains'])

        all_min_dates = []
        try:
            for network, tvls in data_date[protocol_slug]["chainTvls"].items():
                tvl_list = tvls["tvl"]
                min_date = min(entry["date"] for entry in tvl_list)
                all_min_dates.append(min_date)
            min_date = min(all_min_dates) if all_min_dates else None
            min_date = timestamp_to_date(min_date)

            results.append([protocol_name, protocol_address, protocol_chains, protocol_tvl, min_date])

        except Exception as e:
            print(f"Error scrapping data for slug: {protocol_slug}. Error: {e}")
            problematic_slugs.append(protocol_slug)
    
print(f"Total problematic slugs: {len(problematic_slugs)}")
    

with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Name', 'Address', 'Chains', 'TVL', 'Earliest Date'])
    writer.writerows(results)

print("Data saved to output.csv!")


Total problematic slugs: 0
Data saved to output.csv!
